In [ ]:
import heapq
from pathlib import Path
from typing import Literal

type Position = tuple[int, int]
type Direction = Literal["N", "E", "W", "S"]
type State = tuple[Position, Direction]

DIRECTIONS: list[Direction] = ["N", "E", "W", "S"]

In [ ]:
WALLS: set[Position] = set()
facing: Direction = "E"

with Path("day16_input.txt").open() as file:
    for row, line in enumerate(file):
        for col, char in enumerate(line.strip()):
            if char == "S":
                START = (row, col)
            elif char == "E":
                END = (row, col)
            elif char == "#":
                WALLS.add((row, col))

In [ ]:
def neighbors_with_cost(state: State) -> list[tuple[State, int]]:
    """Find possible neighbors with corresponding scores.

    Returns: list of (neighbor, facing, score) tuples.
    """
    (row, col), facing = state
    possible = []
    for direction in DIRECTIONS:
        score = 1 if direction == facing else 1001
        if direction == "N":
            neighbor = (row - 1, col)
        elif direction == "E":
            neighbor = (row, col + 1)
        elif direction == "S":
            neighbor = (row + 1, col)
        elif direction == "W":
            neighbor = (row, col - 1)
        if neighbor not in WALLS:
            possible.append(((neighbor, direction), score))
    return possible

# Part 1


In [ ]:
def find_cost_dijkstra(
    start_state: State, end: Position
) -> tuple[State, int, dict[State, set[State]]]:
    """Use Dijkstras algorithm to find the path with the lowest cost."""
    visited = set()
    previous: dict[State, set[State]] = {start_state: set()}
    scores = {start_state: 0}
    queue = [(0, start_state)]

    while queue:
        # Get the position with the lowest score
        score, state = heapq.heappop(queue)
        if state in visited:
            continue
        visited.add(state)
        # Check if it is better to go through this state
        for new_state, added_score in neighbors_with_cost(state):
            if new_state not in visited:
                new_score = score + added_score
                if new_score < scores.get(new_state, float("inf")):
                    scores[new_state] = new_score
                    previous[new_state] = {state}
                    heapq.heappush(queue, (new_score, new_state))
                if new_score == scores.get(new_state, float("inf")):
                    # Also store equally good path alternatives (part 2)
                    previous[new_state].add(state)

    # The end can be reached from multiple directions, find the best one
    end_scores: dict[State, int] = {
        (end, direction): scores[(end, direction)]
        for direction in DIRECTIONS
        if scores.get((end, direction)) is not None
    }
    solution_state = min(end_scores, key=end_scores.get)

    return solution_state, scores[solution_state], previous

In [ ]:
end_state, score, previous = find_cost_dijkstra((START, facing), END)
end_state, score

# Part 2


In [ ]:
# Backtrack all the previous states from the solution
queue = [end_state]
on_a_best_path = set()
while queue:
    tail = queue.pop()
    on_a_best_path.add(tail[0])
    for prev in previous[tail]:
        queue.append(prev)

In [ ]:
len(on_a_best_path)